In [8]:
# Necessary imports
import numpy as np
import py3Dmol

In [9]:
def printXYZ(coords, filename, step, mode='vmd'):
    """ Write Coordinates to trajectory file """
    nPart=coords.shape[0]
    if mode=='vmd':
        with open(filename+'.xyz', 'a') as xyz:
            xyz.write(f"{nPart}\n\n")
            for i in range(nPart):
                xyz.write(f"Ar      {coords[i][0]:10.5f} {coords[i][1]:10.5f} {coords[i][2]:10.5f}\n")
    if mode=='python':
        with open(filename+'.'+str(step)+'.xyz', 'a') as xyz:
            xyz.write(f"{nPart}\n\n")
            for i in range(nPart):
                xyz.write(f"Ar      {coords[i][0]:10.5f} {coords[i][1]:10.5f} {coords[i][2]:10.5f}\n")        

In [10]:
def placeParticlesOnGrid(nPart=100, density=0.85):
    """Initialize LJ Particles on grid"""
    coords=np.zeros((nPart,3))
    L=(nPart/density)**(1.0/3)
    nCube=2
    
    while (nCube**3 < nPart):
        nCube+=1
    index = np.array([0,0,0])
    
    for part in range(nPart):
        coords[part]=index+np.array([0.5,0.5,0.5])*(L/nCube)
        index[0]+=1
        if index[0]==nCube:
            index[0]=0
            index[1]+=1
            if index[1]==nCube:
                index[1]=0
                index[2]+=1
    return coords, L

In [11]:
def LJ_Energy(coords, L):
    """Calculate energy according to Lennard Jones Potential"""
    energy=0
    
    nPart=coords.shape[0]
    
    for partA in range(nPart-1):
        for partB in range(partA+1,nPart):
            dr = coords[partA]-coords[partB]
            
            dr = distPBC3D(dr, L)
            
            dr2=np.sum(np.dot(dr,dr))
            
            #Lennard-Jones potential:
            # U(r) = 4*epsilon* [(sigma/r)^12 - (sigma/r)^6]
            # Here, we set sigma = 1, epsilon = 1 (reduced distance and energy units). Therefore:
            # U(r) = 4 * [(1/r)^12 - (1/r)^6]
            # For efficiency, we will multiply by 4 only after summing
            # up all the energies.
            
            invDr6= 1.0/(dr2**3) # 1/r^6
            
            energy = energy +(invDr6*(invDr6-1))
    
    return energy*4

In [12]:
def distPBC3D(dr, L):
    """Calculate distance according to minimum image convention"""
    hL=L/2.0
    
    # Distance vector needs to be in [-hLx, hLX], [-hLy, hLy] [-hLz,hLz]
    
    for dim in range(3):
        if dr[dim]>hL:
            dr[dim]-=L
        elif dr[dim]<-hL:
            dr[dim]+=L
            
    return dr

In [13]:
def putParticlesInBox(vec, L):
    """Put coordinates back into primary periodic image"""
    # Coord needs to be in [0, L], [0,L] [0,L]
    
    for dim in range(3):
        if vec[dim]>L:
            vec[dim]-=L
        elif vec[dim]<-L:
            vec[dim]+=L
            
    return vec

In [14]:
def deltaLJ(coords, trialPos, part, L):
    """Calculate Energy change of Move"""
    deltaE=0
    
    npart=coords.shape[0]
    
    for otherPart in range(nPart):
        if otherPart == part:
            continue
        
        # Particle Particle dist
        drNew = coords[otherPart]-trialPos
        drOld = coords[otherPart]-coords[part]
        
        drNew = distPBC3D(drNew, L)
        drOld = distPBC3D(drOld, L)
        
        dr2_New = np.sum(np.dot(drNew, drNew))
        dr2_Old = np.sum(np.dot(drOld, drOld))
        #   Lennard-Jones potential:
        #    U(r) = 4*epsilon* [(sigma/r)^12 - (sigma/r)^6]
        #    with sigma = 1, epsilon = 1 (reduced units). 
        #   => 
        #   U(r) = 4 * [(1/r)^12 - (1/r)^6]
        #   We multiply by 4 only in the end
        
        invDr6_New = 1.0/(dr2_New**3)
        invDr6_Old = 1.0/(dr2_Old**3)
        
        eNew = (invDr6_New*(invDr6_New-1))
        eOld = (invDr6_Old*(invDr6_Old-1))
        
        deltaE = deltaE + eNew-eOld
        
        return deltaE*4


In [17]:
#Set configuration parameters
nPart = 100;        # Number of particles
density = 0.4;    # Density of particles
       
#Set simulation parameters
Temp = 2.0;         # Simulation temperature
beta = 1.0/Temp;    # Inverse temperature
maxDr = 0.1;        # Maximal displacement

nSteps = 10000;     #Total simulation time (in integration steps)
printFreq = 1000   #Printing frequency
        

In [19]:
# Set initial configuration
density = [0.05, 0.20, 0.50, 1] # to perform simulation for different values of density

for density in density:
    print('density: ', density)
    coords, L = placeParticlesOnGrid(nPart,density)#

    # Calculate initial energy
    energy = LJ_Energy(coords,L)

    for step in range(nSteps):
        if step%printFreq==0:
            print(f"MC Step {step:2}  Energy {energy:10.5f}")
            printXYZ(coords, 'trajectory', step, mode='python') # mode VMD
        for i in range(nPart):
            rTrial = coords[i]+maxDr*(np.random.rand(1,3).squeeze()-0.5)
            rTrial = putParticlesInBox(rTrial, L)

            deltaE = deltaLJ(coords, rTrial, i, L)

            if np.random.rand() < np.exp(-beta*deltaE):
                coords[i]=rTrial
                energy +=deltaE
    print('------------------------------------')

density:  0.05
MC Step  0  Energy -230.95720
MC Step 1000  Energy -261.69012
MC Step 2000  Energy -268.68778
MC Step 3000  Energy -268.85784
MC Step 4000  Energy -268.76640
MC Step 5000  Energy -268.73485
MC Step 6000  Energy -268.60572
MC Step 7000  Energy -268.56946
MC Step 8000  Energy -268.56740
MC Step 9000  Energy -268.59328
------------------------------------
density:  0.2
MC Step  0  Energy -230.98794
MC Step 1000  Energy -262.72369
MC Step 2000  Energy -290.02754
MC Step 3000  Energy -427.66277
MC Step 4000  Energy -425.93775
MC Step 5000  Energy -426.98333
MC Step 6000  Energy -431.57421
MC Step 7000  Energy -465.69744
MC Step 8000  Energy -582.27160
MC Step 9000  Energy -680.37732
------------------------------------
density:  0.5
MC Step  0  Energy -246.24139
MC Step 1000  Energy -377.68904
MC Step 2000  Energy -1138.41537
MC Step 3000  Energy -1361.89611
MC Step 4000  Energy -1749.63209
MC Step 5000  Energy -2140.78533
MC Step 6000  Energy -2352.97488
MC Step 7000  Energy

In [22]:
################################
########## QUESTION 3 ##########

#Set configuration parameters
nPart = 100;        # Number of particles
density = 0.4;    # Density of particles
       
#Set simulation parameters
Temp = 2.0;         # Simulation temperature
beta = 1.0/Temp;    # Inverse temperature
maxDr = 0.1;        # Maximal displacement

nSteps = 10000;     #Total simulation time (in integration steps)
printFreq = 1000   #Printing frequency

In [24]:
# Set initial configuration
print('density: ', density)
coords, L = placeParticlesOnGrid(nPart,density)#

# Calculate initial energy
energy = LJ_Energy(coords,L)
for step in range(nSteps):
    if step%printFreq==0:
        print(f"MC Step {step:2}  Energy {energy:10.5f}")
        printXYZ(coords, 'trajectory', step, mode='python') # mode VMD
    for i in range(nPart):
        rTrial = coords[i]+maxDr*(np.random.rand(1,3).squeeze()-0.5)
        rTrial = putParticlesInBox(rTrial, L)

        deltaE = deltaLJ(coords, rTrial, i, L)

        if np.random.rand() < np.exp(-beta*deltaE):
            coords[i]=rTrial
            energy +=deltaE
print('------------------------------------')

density:  0.4
MC Step  0  Energy -235.93251
MC Step 1000  Energy -434.92541
MC Step 2000  Energy -564.13843
MC Step 3000  Energy -810.95997
MC Step 4000  Energy -2015.89708
MC Step 5000  Energy -2126.71837
MC Step 6000  Energy -2259.25606
MC Step 7000  Energy -2564.24150
MC Step 8000  Energy -2923.41159
MC Step 9000  Energy -3058.62743
------------------------------------


In [26]:
view = py3Dmol.view(data="",linked=False,viewergrid=(2,2))

with open('trajectory.0.xyz', 'r') as file:
    xyz = file.read()
view.addModel(xyz,  viewer=(0,0))
view.setStyle({'sphere':{'scale':0.2}}, viewer=(0,0))
view.zoomTo(viewer=(0,0))


with open('trajectory.1000.xyz', 'r') as file:
    xyz = file.read()
view.addModel(xyz,  viewer=(0,1))
view.setStyle({'sphere':{'scale':0.2}}, viewer=(0,1))
view.zoomTo(viewer=(0,1))

with open('trajectory.5000.xyz', 'r') as file:
    xyz = file.read()
view.addModel(xyz,  viewer=(1,0))
view.setStyle({'sphere':{'scale':0.2}}, viewer=(1,0))
view.zoomTo(viewer=(1,0))

with open('trajectory.9000.xyz', 'r') as file:
    xyz = file.read()
view.addModel(xyz,  viewer=(1,1))
view.setStyle({'sphere':{'scale':0.2}}, viewer=(1,1))
view.zoomTo(viewer=(1,1))

view.render()
view.sav

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

####### MC NPT Program #######

In [33]:
#Initialize
#===================
    
#Set configuration parameters
nPart = 100        # Number of particles
#density = 0.85    # Density of particles
density = 10
    
#Set simulation parameters
Temp = 2.0         # Simulation temperature
beta = 1.0/Temp    # Inverse temperature
maxDr = 0.1        # Maximal displacement
press = 1.0
maxDv = 0.01

nSteps = 1000;     #Total simulation time (in integration steps)
printFreq = 100   #Printing frequency
sampleCounter = 0
sampleFreq = 100        

In [35]:
# Set initial configuration
coords, L = placeParticlesOnGrid(nPart,density)
        
# Calculate initial energy
energy = LJ_Energy(coords,L)

avgL = 0

for step in range(nSteps):
    
    # Choose whether to do a volume move or a displacement move
    
    if (np.random.rand()*(nPart+1)+1 < nPart):
        for i in range(nPart):
            rTrial = coords[i]+maxDr*(np.random.rand(1,3).squeeze()-0.5)
            rTrial = putParticlesInBox(rTrial, L)

            deltaE = deltaLJ(coords, rTrial, i, L)

            if np.random.rand() < np.exp(-beta*deltaE):
                coords[i]=rTrial
                energy +=deltaE
    else:
        oldV= L**3
        
        lnvTrial = np.log(oldV)+(np.random.rand()-0.5)*maxDv
        vTrial = np.exp(lnvTrial)
        newL = vTrial**(1.0/3)
        
        # Rescale coords
        coordsTrial=coords*(newL/L)
        
        eTrial = LJ_Energy(coordsTrial, newL)
        
        weight = (eTrial - energy) + press*(vTrial-oldV) -  (nPart+1)*Temp*np.log(vTrial/oldV)
        
        if (np.random.rand()<np.exp(-beta*weight)):
            coords = coordsTrial
            energy = eTrial
            L = newL
    if step%printFreq==0:
        print(f"MC Step {step:2}  Energy {energy:10.5f}")
        printXYZ(coords, 'npt_trajectory', step, mode='python')
    if step%sampleFreq==0:
        sampleCounter+=1
        avgL+=L
        print(f"Average Box Size {avgL/sampleCounter:3.3f}")
        

/home/bruno/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: overflow encountered in exp


MC Step  0  Energy 3496824060260.48779
Average Box Size 0.196
MC Step 100  Energy 3119394846905.07031
Average Box Size 0.359
MC Step 200  Energy 3119394845726.75244
Average Box Size 0.497
MC Step 300  Energy 3119394844510.64453
Average Box Size 0.616
MC Step 400  Energy 3119394842619.60205
Average Box Size 0.718
MC Step 500  Energy 3119394842285.29980
Average Box Size 0.808
MC Step 600  Energy 3119394841783.83984
Average Box Size 0.887
MC Step 700  Energy 3119394840800.93066
Average Box Size 0.958
MC Step 800  Energy 3119394840285.46729
Average Box Size 1.021
MC Step 900  Energy 3119394839929.37305
Average Box Size 1.077
